<a href="https://colab.research.google.com/github/codeslayr/Neural_Network_Optimization/blob/main/Advance_Opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Optimizing CNN hyperparameters using GA and PSO**

Baseline CNN

In [5]:
# Step 1: Imports & Setup
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Step 2: Data Loading
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts to [0, 1]
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Using device: cuda


In [6]:
# Step 3: Define CNN Model
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # output: (32, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # output: (32, 14, 14)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # output: (64, 14, 14)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # output: (64, 7, 7)

            nn.Flatten(),  # output: (64*7*7,)
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)

model = FashionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")

def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    loss_total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss_total += loss.item()

    acc = 100 * correct / total
    avg_loss = loss_total / len(dataloader)
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {acc:.2f}%")
    return avg_loss, acc

In [8]:
train_model(model, train_loader, criterion, optimizer, epochs=5)
evaluate_model(model, test_loader)

Epoch [1/5], Loss: 408.0778, Accuracy: 84.32%
Epoch [2/5], Loss: 263.7032, Accuracy: 89.76%
Epoch [3/5], Loss: 218.7325, Accuracy: 91.42%
Epoch [4/5], Loss: 189.3577, Accuracy: 92.58%
Epoch [5/5], Loss: 167.2148, Accuracy: 93.36%
Test Loss: 0.2429, Test Accuracy: 91.36%


(0.24288778877846753, 91.36)

GA on CNN  

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import random
import numpy as np
from copy import deepcopy

# GPU Config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST('./data', train=True, download=True, transform=transform)
val_size = 5000
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [len(train_dataset)-val_size, val_size])
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_subset, batch_size=64, shuffle=False)

# CNN Model Builder
class CNN(nn.Module):
    def __init__(self, conv1_out, conv2_out, fc_units):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, conv1_out, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(conv1_out, conv2_out, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten_size = (conv2_out * 7 * 7)
        self.fc1 = nn.Linear(self.flatten_size, fc_units)
        self.fc2 = nn.Linear(fc_units, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.flatten_size)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# GA Hyperparameters
POP_SIZE = 20
GENERATIONS = 10
TOP_K = 4

param_bounds = {
    "lr": [1e-4, 1e-2],
    "conv1_out": [16, 64],
    "conv2_out": [32, 128],
    "fc_units": [64, 256],
}

def random_gene():
    return {
        "lr": round(10 ** random.uniform(np.log10(param_bounds['lr'][0]), np.log10(param_bounds['lr'][1])), 5),
        "conv1_out": random.randint(*param_bounds['conv1_out']),
        "conv2_out": random.randint(*param_bounds['conv2_out']),
        "fc_units": random.randint(*param_bounds['fc_units'])
    }

def mutate(gene):
    mutated = deepcopy(gene)
    key = random.choice(list(param_bounds.keys()))
    return random_gene() if random.random() < 0.3 else gene

def crossover(parent1, parent2):
    child = {}
    for key in param_bounds:
        child[key] = parent1[key] if random.random() < 0.5 else parent2[key]
    return child

# Fitness Evaluation
def evaluate(gene):
    model = CNN(gene['conv1_out'], gene['conv2_out'], gene['fc_units']).to(device)
    optimizer = optim.Adam(model.parameters(), lr=gene['lr'])
    criterion = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(1):  # train only 1 epoch
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    model.eval()
    val_loss, correct = 0.0, 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader)
    accuracy = correct / len(val_loader.dataset)
    return val_loss, accuracy

# Genetic Algorithm Core
def run_ga():
    population = [random_gene() for _ in range(POP_SIZE)]

    # Create history storage
    ga_history = []

    best_overall = None
    best_loss = float('inf')
    best_acc = 0.0
    best_gen = -1

    for gen in range(GENERATIONS):
        # Evaluate all individuals
        scored = []
        for gene in population:
            loss, acc = evaluate(gene)
            scored.append((gene, loss, acc))  # <-- MODIFIED TO STORE TUPLE

        scored.sort(key=lambda x: x[1])  # sort by loss

        best_gene, loss, acc = scored[0]

        # Track global best
        if loss < best_loss:
            best_loss = loss
            best_acc = acc
            best_overall = best_gene
            best_gen = gen

        print(f"Gen {gen}: Best Loss = {loss:.4f}, Acc = {acc:.4f}")
        print(f"         Best Hyperparams @ Gen {gen}: {best_gene}")

        # ======== STORE GENERATION HISTORY ======== <-- ADD THIS BLOCK
        gen_data = {
            'generation': gen,
            'population': [gene for gene, _, _ in scored],
            'fitness': [(loss, acc) for _, loss, acc in scored],
            'best_gene': best_gene,
            'best_loss': loss,
            'best_acc': acc
        }
        ga_history.append(gen_data)
        # ======== END HISTORY STORAGE ========

        # Genetic operations
        top_genes = [x[0] for x in scored[:TOP_K]]
        new_pop = top_genes.copy()

        while len(new_pop) < POP_SIZE:
            p1, p2 = random.sample(top_genes, 2)
            child = crossover(p1, p2)
            child = mutate(child)
            new_pop.append(child)

        population = new_pop

    print(f"\n✅ Best hyperparameters found at generation {best_gen}: {best_overall}")
    print(f"   Final Best Loss = {best_loss:.4f}, Accuracy = {best_acc:.4f}")
    return best_overall, ga_history

# Run GA
best_hyperparams, ga_history = run_ga()
print("\nBest hyperparameters found:", best_hyperparams)

KeyboardInterrupt: 

In [ ]:
# Load Official Test Dataset
test_dataset = datasets.FashionMNIST('./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate Best Hyperparams on Test Set
def train_and_test(hparams, epochs=5):
    model = CNN(hparams['conv1_out'], hparams['conv2_out'], hparams['fc_units']).to(device)
    optimizer = optim.Adam(model.parameters(), lr=hparams['lr'])
    criterion = nn.CrossEntropyLoss()

    print(f"\n🔁 Training final model for {epochs} epochs...")
    for epoch in range(epochs):
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1} completed.")

    # Test
    model.eval()
    test_loss, correct = 0.0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    accuracy = correct / len(test_loader.dataset)

    print(f"\n📊 Final Evaluation on Test Set:")
    print(f"   Test Loss     = {test_loss:.4f}")
    print(f"   Test Accuracy = {accuracy:.4f}")

# Evaluate on test set
train_and_test(best_hyperparams)

PSO on CNN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import random
import numpy as np
from copy import deepcopy

# ------------------------------
# Device Setup
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------------
# Dataset (Subset for Speed)
# ------------------------------
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.FashionMNIST('./data', train=True, download=True, transform=transform)
val_size = 5000
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [len(train_dataset) - val_size, val_size])
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_subset, batch_size=64, shuffle=False)

# Test Set (Official Evaluation)
test_dataset = datasets.FashionMNIST('./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# ------------------------------
# CNN Definition
# ------------------------------
class CNN(nn.Module):
    def __init__(self, conv1_out, conv2_out, fc_units):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, conv1_out, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(conv1_out, conv2_out, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten_size = conv2_out * 7 * 7
        self.fc1 = nn.Linear(self.flatten_size, fc_units)
        self.fc2 = nn.Linear(fc_units, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.flatten_size)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# ------------------------------
# Hyperparameter Bounds
# ------------------------------
param_bounds = {
    'lr': [1e-4, 1e-2],
    'conv1_out': [16, 64],
    'conv2_out': [32, 128],
    'fc_units': [64, 256]
}

# ------------------------------
# Fitness Function
# ------------------------------
def evaluate(hparams):
    model = CNN(int(hparams['conv1_out']), int(hparams['conv2_out']), int(hparams['fc_units'])).to(device)
    optimizer = optim.Adam(model.parameters(), lr=hparams['lr'])
    criterion = nn.CrossEntropyLoss()

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx >= 9:  # train on 10 mini-batches
            break

    model.eval()
    val_loss, correct = 0.0, 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

    val_loss /= len(val_loader)
    accuracy = correct / len(val_loader.dataset)
    return val_loss, accuracy

# ------------------------------
# PSO Core
# ------------------------------
def run_pso(particles=10, generations=20, w=0.5, c1=1.5, c2=1.5):
    def init_particle():
        return {
            'lr': 10 ** random.uniform(np.log10(param_bounds['lr'][0]), np.log10(param_bounds['lr'][1])),
            'conv1_out': random.randint(*param_bounds['conv1_out']),
            'conv2_out': random.randint(*param_bounds['conv2_out']),
            'fc_units': random.randint(*param_bounds['fc_units'])
        }

    def clip(hp):
        hp['lr'] = float(np.clip(hp['lr'], *param_bounds['lr']))
        hp['conv1_out'] = int(np.clip(round(hp['conv1_out']), *param_bounds['conv1_out']))
        hp['conv2_out'] = int(np.clip(round(hp['conv2_out']), *param_bounds['conv2_out']))
        hp['fc_units'] = int(np.clip(round(hp['fc_units']), *param_bounds['fc_units']))
        return hp

    population = [init_particle() for _ in range(particles)]
    velocity = [{k: 0.0 for k in p} for p in population]

    personal_best = deepcopy(population)
    personal_best_scores = []
    personal_best_accs = []

    for p in personal_best:
        loss, acc = evaluate(p)
        personal_best_scores.append(loss)
        personal_best_accs.append(acc)

    global_best_idx = np.argmin(personal_best_scores)
    global_best = deepcopy(personal_best[global_best_idx])
    global_best_loss = personal_best_scores[global_best_idx]
    global_best_acc = personal_best_accs[global_best_idx]

    pso_history = []

    for gen in range(generations):
        for i in range(particles):
            new_vel = {}
            new_pos = {}
            for key in population[i]:
                r1, r2 = random.random(), random.random()
                vel = (w * velocity[i][key]
                       + c1 * r1 * (personal_best[i][key] - population[i][key])
                       + c2 * r2 * (global_best[key] - population[i][key]))
                new_vel[key] = vel
                new_pos[key] = population[i][key] + vel
            velocity[i] = new_vel
            new_pos = clip(new_pos)
            population[i] = new_pos

            loss, acc = evaluate(new_pos)
            if loss < personal_best_scores[i]:
                personal_best[i] = deepcopy(new_pos)
                personal_best_scores[i] = loss
                personal_best_accs[i] = acc

        global_best_idx = np.argmin(personal_best_scores)
        global_best = deepcopy(personal_best[global_best_idx])
        global_best_loss = personal_best_scores[global_best_idx]
        global_best_acc = personal_best_accs[global_best_idx]

        print(f"Gen {gen}: Best Loss = {global_best_loss:.4f}, Acc = {global_best_acc:.4f}")

        # ======== STORE ITERATION HISTORY ======== <-- ADD THIS BLOCK
        gen_data = {
            'iteration': gen,
            'swarm': deepcopy(population),
            'fitness': [(personal_best_scores[i], personal_best_accs[i])
                        for i in range(len(population))],
            'global_best': deepcopy(global_best),
            'global_loss': global_best_loss,
            'global_acc': global_best_acc
        }
        pso_history.append(gen_data)
        # ======== END HISTORY STORAGE ========

    return global_best, pso_history

# ------------------------------
# Run PSO
# ------------------------------
best_pso_hyperparams, pso_history = run_pso()
print("\nBest hyperparameters from PSO:", best_pso_hyperparams)

# ------------------------------
# Final Full Training & Test Evaluation
# ------------------------------
def train_and_test(hparams, epochs=5):
    model = CNN(hparams['conv1_out'], hparams['conv2_out'], hparams['fc_units']).to(device)
    optimizer = optim.Adam(model.parameters(), lr=hparams['lr'])
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1} completed.")

    model.eval()
    test_loss, correct = 0.0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

    test_loss /= len(test_loader)
    test_acc = correct / len(test_loader.dataset)
    print(f"\nTest Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

train_and_test(best_pso_hyperparams)

### **Plotting of GA vs PSO**

In [ ]:
# ==================================================================
# PLOTTING FUNCTIONS
# ==================================================================

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from pandas.plotting import parallel_coordinates
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go

# Normalization helper
def normalize_param(value, param_name):
    bounds = param_bounds[param_name]
    if param_name == 'lr':
        log_val = np.log10(value)
        log_min = np.log10(bounds[0])
        log_max = np.log10(bounds[1])
        return (log_val - log_min) / (log_max - log_min)
    return (value - bounds[0]) / (bounds[1] - bounds[0])

In [ ]:
# 1. CONVERGENCE PLOT
def plot_convergence(ga_history, pso_history):
    plt.figure(figsize=(12, 6))

    # GA data
    ga_loss = [gen['best_loss'] for gen in ga_history]
    ga_acc = [gen['best_acc'] for gen in ga_history]

    # PSO data
    pso_loss = [gen['global_loss'] for gen in pso_history]
    pso_acc = [gen['global_acc'] for gen in pso_history]

    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(ga_loss, 'b-o', label='GA')
    plt.plot(pso_loss, 'r-s', label='PSO')
    plt.xlabel('Generation/Iteration')
    plt.ylabel('Validation Loss')
    plt.title('Loss Convergence')
    plt.legend()
    plt.grid(True)

    # Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(ga_acc, 'b-o', label='GA')
    plt.plot(pso_acc, 'r-s', label='PSO')
    plt.xlabel('Generation/Iteration')
    plt.ylabel('Validation Accuracy')
    plt.title('Accuracy Convergence')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('convergence.png', dpi=300)
    plt.show()

plot_convergence(ga_history, pso_history)

In [ ]:
# 2. 3D TRAJECTORY ANIMATION
def create_3d_animation(ga_history, pso_history):
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Prepare data containers
    ga_points = []
    pso_points = []

    # Create animation function
    def update(frame):
        ax.clear()

        # GA points
        for gen in range(min(frame+1, len(ga_history))):
            for gene in ga_history[gen]['population']:
                x = normalize_param(gene['lr'], 'lr')
                y = normalize_param(gene['conv1_out'], 'conv1_out')
                z = normalize_param(gene['conv2_out'], 'conv2_out')
                ga_points.append((x, y, z))

        # PSO points
        for iter in range(min(frame+1, len(pso_history))):
            for particle in pso_history[iter]['swarm']:
                x = normalize_param(particle['lr'], 'lr')
                y = normalize_param(particle['conv1_out'], 'conv1_out')
                z = normalize_param(particle['conv2_out'], 'conv2_out')
                pso_points.append((x, y, z))

        # Convert to arrays
        if ga_points:
            ga_arr = np.array(ga_points)
            ax.scatter(ga_arr[:,0], ga_arr[:,1], ga_arr[:,2],
                      c='blue', alpha=0.3, s=10, label='GA')

        if pso_points:
            pso_arr = np.array(pso_points)
            ax.scatter(pso_arr[:,0], pso_arr[:,1], pso_arr[:,2],
                      c='red', alpha=0.3, s=10, marker='s', label='PSO')

        # Plot best trajectories
        if ga_history:
            ga_best = []
            for gen in range(min(frame+1, len(ga_history))):
                gene = ga_history[gen]['best_gene']
                x = normalize_param(gene['lr'], 'lr')
                y = normalize_param(gene['conv1_out'], 'conv1_out')
                z = normalize_param(gene['conv2_out'], 'conv2_out')
                ga_best.append((x, y, z))
            ga_best_arr = np.array(ga_best)
            ax.plot(ga_best_arr[:,0], ga_best_arr[:,1], ga_best_arr[:,2],
                   'b-', linewidth=2, label='GA Best')
            ax.scatter(ga_best_arr[-1,0], ga_best_arr[-1,1], ga_best_arr[-1,2],
                      c='gold', s=100, edgecolor='black', label='Current Best')

        if pso_history:
            pso_best = []
            for iter in range(min(frame+1, len(pso_history))):
                particle = pso_history[iter]['global_best']
                x = normalize_param(particle['lr'], 'lr')
                y = normalize_param(particle['conv1_out'], 'conv1_out')
                z = normalize_param(particle['conv2_out'], 'conv2_out')
                pso_best.append((x, y, z))
            pso_best_arr = np.array(pso_best)
            ax.plot(pso_best_arr[:,0], pso_best_arr[:,1], pso_best_arr[:,2],
                   'r-', linewidth=2, label='PSO Best')

        # Set labels and title
        ax.set_xlabel('log10(Learning Rate)')
        ax.set_ylabel('conv1_out')
        ax.set_zlabel('conv2_out')
        ax.set_title(f'Hyperparameter Optimization Trajectories\nGeneration/Iteration: {frame}')
        ax.legend()

        # Set viewing angle
        ax.view_init(30, frame * 2)

    # Create animation
    ani = FuncAnimation(fig, update, frames=max(len(ga_history), len(pso_history)),
                        interval=500, blit=False)

    # Save animation
    ani.save('optimization_trajectory.mp4', writer='ffmpeg', fps=2, dpi=150)
    plt.close()

create_3d_animation(ga_history, pso_history)

In [ ]:
# 3. PARALLEL COORDINATES PLOT
def plot_parallel_coordinates(ga_history, pso_history):
    data = []

    # Collect GA data
    for gen in ga_history:
        for gene, (loss, acc) in zip(gen['population'], gen['fitness']):
            row = gene.copy()
            row['loss'] = loss
            row['accuracy'] = acc
            row['method'] = 'GA'
            data.append(row)

    # Collect PSO data
    for gen in pso_history:
        for particle, (loss, acc) in zip(gen['swarm'], gen['fitness']):
            row = particle.copy()
            row['loss'] = loss
            row['accuracy'] = acc
            row['method'] = 'PSO'
            data.append(row)

    # Create DataFrame
    df = pd.DataFrame(data)

    # Normalize parameters for better visualization
    for param in param_bounds.keys():
        if param == 'lr':
            df['log_lr'] = np.log10(df['lr'])
            min_val = np.log10(param_bounds['lr'][0])
            max_val = np.log10(param_bounds['lr'][1])
            df['norm_lr'] = (df['log_lr'] - min_val) / (max_val - min_val)
        else:
            min_val = param_bounds[param][0]
            max_val = param_bounds[param][1]
            df[f'norm_{param}'] = (df[param] - min_val) / (max_val - min_val)

    plt.figure(figsize=(14, 8))
    parallel_coordinates(
        df[['method', 'norm_lr', 'norm_conv1_out', 'norm_conv2_out', 'norm_fc_units', 'accuracy']],
        'method',
        color=['blue', 'red'],
        alpha=0.1
    )
    plt.title('Hyperparameter Space Exploration')
    plt.xlabel('Parameters (Normalized)')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.grid(alpha=0.3)
    plt.savefig('parallel_coordinates.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_parallel_coordinates(ga_history, pso_history)

In [ ]:
# 4. HYPERPARAMETER EVOLUTION
def plot_hyperparameter_evolution(ga_history, pso_history):
    fig, axs = plt.subplots(2, 2, figsize=(16, 12))
    params = ['lr', 'conv1_out', 'conv2_out', 'fc_units']

    for i, param in enumerate(params):
        ax = axs[i//2, i%2]

        # GA evolution
        ga_vals = []
        for gen in ga_history:
            gen_vals = [gene[param] for gene in gen['population']]
            ga_vals.append(gen_vals)
        ga_vals = np.array(ga_vals)

        # Plot GA
        for j in range(ga_vals.shape[1]):
            ax.plot(ga_vals[:, j], 'b-', alpha=0.1)
        ax.plot(np.median(ga_vals, axis=1), 'b-o', linewidth=2, label='GA Median')

        # PSO evolution
        pso_vals = []
        for gen in pso_history:
            gen_vals = [particle[param] for particle in gen['swarm']]
            pso_vals.append(gen_vals)
        pso_vals = np.array(pso_vals)

        # Plot PSO
        for j in range(pso_vals.shape[1]):
            ax.plot(pso_vals[:, j], 'r-', alpha=0.1)
        ax.plot(np.median(pso_vals, axis=1), 'r-s', linewidth=2, label='PSO Median')

        # Formatting
        ax.set_xlabel('Generation/Iteration')
        ax.set_ylabel(param)
        if param == 'lr':
            ax.set_yscale('log')
        ax.set_title(f'{param} Evolution')
        ax.legend()
        ax.grid(True)

    plt.tight_layout()
    plt.savefig('hyperparameter_evolution.png', dpi=300)
    plt.show()

plot_hyperparameter_evolution(ga_history, pso_history)

In [ ]:
# 5. PERFORMANCE HEATMAP
def plot_performance_heatmap(ga_history, pso_history):
    # Prepare data
    all_data = []

    # GA data
    for gen in ga_history:
        for gene, (loss, acc) in zip(gen['population'], gen['fitness']):
            all_data.append({
                'method': 'GA',
                'conv1_out': gene['conv1_out'],
                'conv2_out': gene['conv2_out'],
                'accuracy': acc
            })

    # PSO data
    for gen in pso_history:
        for particle, (loss, acc) in zip(gen['swarm'], gen['fitness']):
            all_data.append({
                'method': 'PSO',
                'conv1_out': particle['conv1_out'],
                'conv2_out': particle['conv2_out'],
                'accuracy': acc
            })

    df = pd.DataFrame(all_data)

    # Create pivot tables
    ga_pivot = df[df['method']=='GA'].pivot_table(
        index='conv1_out', columns='conv2_out', values='accuracy', aggfunc='mean'
    )
    pso_pivot = df[df['method']=='PSO'].pivot_table(
        index='conv1_out', columns='conv2_out', values='accuracy', aggfunc='mean'
    )

    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7), sharey=True)

    # GA heatmap
    sns.heatmap(ga_pivot, annot=True, fmt=".3f", cmap="Blues", ax=ax1)
    ax1.set_title('GA Accuracy Heatmap')
    ax1.set_xlabel('conv2_out')
    ax1.set_ylabel('conv1_out')

    # PSO heatmap
    sns.heatmap(pso_pivot, annot=True, fmt=".3f", cmap="Reds", ax=ax2)
    ax2.set_title('PSO Accuracy Heatmap')
    ax2.set_xlabel('conv2_out')

    plt.tight_layout()
    plt.savefig('performance_heatmap.png', dpi=300)
    plt.show()

plot_performance_heatmap(ga_history, pso_history)

In [ ]:
# 6. INTERACTIVE 3D PLOT (PLOTLY)
def create_interactive_3d(ga_history, pso_history):
    # Prepare data
    ga_data = []
    pso_data = []

    # GA points
    for gen in ga_history:
        for gene, (loss, acc) in zip(gen['population'], gen['fitness']):
            ga_data.append({
                'x': np.log10(gene['lr']),
                'y': gene['conv1_out'],
                'z': gene['conv2_out'],
                'accuracy': acc,
                'fc_units': gene['fc_units'],
                'generation': gen['generation']
            })

    # PSO points
    for gen in pso_history:
        for particle, (loss, acc) in zip(gen['swarm'], gen['fitness']):
            pso_data.append({
                'x': np.log10(particle['lr']),
                'y': particle['conv1_out'],
                'z': particle['conv2_out'],
                'accuracy': acc,
                'fc_units': particle['fc_units'],
                'iteration': gen['iteration']
            })

    # Create GA trace
    ga_df = pd.DataFrame(ga_data)
    ga_trace = go.Scatter3d(
        x=ga_df['x'],
        y=ga_df['y'],
        z=ga_df['z'],
        mode='markers',
        marker=dict(
            size=ga_df['fc_units']/50,
            color=ga_df['accuracy'],
            colorscale='Blues',
            opacity=0.7,
            colorbar=dict(title='Accuracy')
        ),
        name='GA',
        hovertext=ga_df.apply(lambda row:
            f"Accuracy: {row['accuracy']:.4f}<br>"
            f"LR: {10**row['x']:.5f}<br>"
            f"conv1: {row['y']}<br>"
            f"conv2: {row['z']}<br>"
            f"fc: {row['fc_units']}<br>"
            f"Gen: {row['generation']}", axis=1)
    )

    # Create PSO trace
    pso_df = pd.DataFrame(pso_data)
    pso_trace = go.Scatter3d(
        x=pso_df['x'],
        y=pso_df['y'],
        z=pso_df['z'],
        mode='markers',
        marker=dict(
            size=pso_df['fc_units']/50,
            color=pso_df['accuracy'],
            colorscale='Reds',
            opacity=0.7,
            colorbar=dict(title='Accuracy')
        ),
        name='PSO',
        hovertext=pso_df.apply(lambda row:
            f"Accuracy: {row['accuracy']:.4f}<br>"
            f"LR: {10**row['x']:.5f}<br>"
            f"conv1: {row['y']}<br>"
            f"conv2: {row['z']}<br>"
            f"fc: {row['fc_units']}<br>"
            f"Iter: {row['iteration']}", axis=1)
    )

    # Create figure
    fig = go.Figure(data=[ga_trace, pso_trace])

    # Set layout
    fig.update_layout(
        title='Hyperparameter Optimization Space',
        scene=dict(
            xaxis_title='log10(Learning Rate)',
            yaxis_title='conv1_out',
            zaxis_title='conv2_out',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        width=1200,
        height=800
    )

    # Save and show
    fig.write_html("interactive_optimization.html")
    fig.show()

create_interactive_3d(ga_history, pso_history)